In [10]:
# --- Imports ---
import yaml
import json
import numpy as np
import pickle
from pipeline.rag_pipeline import RAGPipeline
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns


ModuleNotFoundError: No module named 'pipeline'

In [11]:
# --- Load config ---
cfg = yaml.safe_load(open('config/config.yaml', 'r', encoding='utf-8'))


FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'

In [ ]:
# --- Initialize RAG pipeline ---
rag_pipeline = RAGPipeline(cfg)


In [ ]:
# --- Load test questions and ground truth ---
with open('evaluation/test_questions.json', 'r', encoding='utf-8') as f:
    test_questions = json.load(f)

with open('evaluation/ground_truth.json', 'r', encoding='utf-8') as f:
    ground_truth = json.load(f)

print(f"Loaded {len(test_questions)} test questions")


In [ ]:
# --- Test pipeline on a single query ---
query = test_questions[0]  # Lấy câu hỏi đầu tiên để thử
answer, top_chunks = rag_pipeline.run(query, return_chunks=True, top_k=cfg.get('pipeline', {}).get('top_k_display', 5))

print("Query:", query)
print("Answer:", answer)
print("Top Chunks:")
for i, chunk in enumerate(top_chunks):
    print(f"Chunk {i+1}: {chunk['text'][:200]}...")  # Hiển thị 200 ký tự đầu


In [ ]:
# --- Evaluate pipeline on all test questions ---
results = []

for q in test_questions:
    answer, _ = rag_pipeline.run(q, return_chunks=True, top_k=cfg.get('pipeline', {}).get('top_k_display', 5))
    gt = ground_truth.get(q, "")
    results.append({
        "query": q,
        "answer": answer,
        "ground_truth": gt
    })

print(f"Evaluated {len(results)} queries")


In [ ]:
# --- Simple Exact Match score ---
exact_matches = sum(1 for r in results if r['answer'].strip() == r['ground_truth'].strip())
accuracy = exact_matches / len(results) if results else 0
print(f"Exact Match Accuracy: {accuracy*100:.2f}%")


In [ ]:
# --- Visualize embeddings of first 100 chunks ---
embeddings_file = cfg.get('paths', {}).get('embeddings_file', 'embeddings/bge_embeddings.npy')
with open(embeddings_file, 'rb') as f:
    embeddings = np.load(f)

sample_embeddings = embeddings[:100]
# Cosine similarity heatmap
sim_matrix = cosine_similarity(sample_embeddings)
plt.figure(figsize=(8,6))
sns.heatmap(sim_matrix, cmap='viridis')
plt.title("Cosine similarity between first 100 chunks")
plt.show()


In [ ]:
# --- Test FAISS index search ---
index_file = cfg.get('paths', {}).get('faiss_index_file', 'faiss_index/index.faiss')
mapping_file = cfg.get('paths', {}).get('mapping_file', 'faiss_index/mapping.pkl')

import faiss

# Load index
index = faiss.read_index(index_file)

# Load mapping
with open(mapping_file, 'rb') as f:
    mapping = pickle.load(f)

query_vec = rag_pipeline.embedding_model.encode([test_questions[0]], convert_to_numpy=True)
D, I = index.search(query_vec, 5)
print("Top 5 retrieved chunks via FAISS:")
for idx in I[0]:
    chunk = mapping.get(idx, {})
    print(chunk.get("text", "")[:200], "...")
